In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

 
driver.get("https://www.bbc.com/news")
print(driver.title)

Home - BBC News


In [3]:
news_urls=[]
def get_urls():
 elements_for_url=driver.find_elements(By.CLASS_NAME, 'gs-c-promo-heading')
 for element in elements_for_url:
        news_urls.append(element.get_attribute('href'))
 print(len(elements_for_url))
get_urls()
print(len(news_urls))#list with all the urls to the articels. should be equally long to elements_for_url
print(news_urls[12])

75
75
https://www.bbc.com/news/uk-67436126


In [4]:
#here we extract all the text contents from the articles. This takes approximately 2 minutes with a total of 76 articels today
def extract_contents(text_contents):
  try:
   article = driver.find_element(By.TAG_NAME, 'article')
   text_contents.append(article.text)
  except: #exception if no article element is there. doesn´t happen often, but has to be handled
   article = driver.find_element(By.TAG_NAME, 'body')
   text_contents.append(article.text)
   print("article not found") #in this test, 3 of 73 articels had no article element. 
#extract_contents()
#print(len(text_contents)) 

In [5]:

#here are a few exceptions, because the bbc articles have different html structures
def get_headers(headers):
  try:
   header=driver.find_element(By.ID, 'main-heading')
   headers.append(header.text)
  except:
   try:
     header = driver.find_element(By.CLASS_NAME, 'qa-story-headline')
     headers.append(header.text)
   except:
    try:
     header = driver.find_element(By.CLASS_NAME, 'article-headline__text')
     headers.append(header.text)
    except:
     header=None
     headers.append(header)
     print('no header')
#get_headers()




In [6]:
def get_timestamps(time):
  try:
   time_element=driver.find_element(By.TAG_NAME, 'time')
   date=(time_element.get_attribute('datetime'))
   time.append(date.split(".")[0])#ignore the error, it works when it´s accessed by the main function
  except:
    date=None
    time.append(date)
    print('no date')

In [7]:

def get_Image(imageURL,ImageDesc):
   try:
    image_element=driver.find_element(By.TAG_NAME, 'img')
    url=(image_element.get_attribute('src'))
    desc=(image_element.get_attribute('alt'))
    imageURL.append(url)
    ImageDesc.append(desc)
   except:
    url=None
    desc=None
    imageURL.append(url)
    ImageDesc.append(desc)
    print('no image')

In [8]:
#main method to extract all contents except the authors(might follow later). Runs way faster than the different smaller methods before
text_contents=[]
time=[]
headers=[]
imageURL=[]
imageDesc=[]
def extract_all():
    for i in range(len(news_urls)):
       driver.get(news_urls[i]) #we go to the specific article by using the url
       extract_contents(text_contents)
       get_timestamps(time)
       get_headers(headers)
       get_Image(imageURL,imageDesc)
extract_all()
print(text_contents[10])
print(time[10])
print(headers[10])
print(imageURL[10])
print(imageDesc[10])

no date
no header
article not found
article not found
no date
article not found
no date
no header
no date
no date
no date
no date
no date
no date
no date
no header
David Cameron makes first official visit to Ukraine
Published
5 hours ago
Share
Related Topics
Russia-Ukraine war
IMAGE SOURCE,
@ZELENSKYYUA
By Jacqueline Howard
BBC News
New Foreign Secretary David Cameron has made his first working visit to Ukraine, meeting President Volodymyr Zelensky in Kyiv.
Lord Cameron reiterated the UK's support for Ukraine, promising moral, diplomatic and "above all military support for... however long it takes".
Mr Zelensky congratulated Mr Cameron on his new post and thanked him for the UK's continued backing.
Lord Cameron became foreign secretary in a cabinet shuffle on Monday.
"This is very important, especially now, when the world is paying attention not only to the situation on the battlefield in Ukraine, and dividing the focus really does not help," the Ukrainian president said, alluding to t

In [9]:
for item in headers:
  print(item)
print(len(headers))
counter=0
for i in range(len(headers)):
    if headers[i]==None:
      counter=counter+1
print(counter)
for item in news_urls:
  print(item)

None
BBC goes inside Al-Shifa hospital with the Israeli army
Gaza faces communications blackout due to lack of fuel
Ukraine war: Russian artist Sasha Skochilenko jailed for anti-war messages
Spain's Pedro Sánchez wins new term as PM after amnesty deal
The Crown: Netflix's royal drama featuring Diana's ghost splits critics
US and China agree to resume military communications after summit
George Santos won't run for re-election after damning ethics report released
Finland to close some crossing points on Russia border - PM
Travis Scott breaks silence on Astroworld tragedy
David Cameron makes first official visit to Ukraine
Travis Scott breaks silence on Astroworld tragedy
David Cameron makes first official visit to Ukraine
Dog survives 10 weeks in Colorado mountains beside dead owner
Russian Carlsberg staff arrested after business seized
Las Vegas teen killed: Two suspects sought in deadly brawl
How the dead are counted in Gaza
Bowen: Al-Shifa Hospital raid comes as tone shifts on Israel

In [10]:
def get_authors(article):
    authors = []
    lines = article.split('\n')
    for line in lines[:13]:  # Loop through lines from lines[0] to lines[10]
        if "By" in line:
            authors.append(line.replace('By', ''))
    return authors

all_authors = []
def get_all_authors():
    for article in text_contents:
        authors = get_authors(article)
        if not authors:  # Check if authors is an empty list
            all_authors.append(None)
        else:
            all_authors.extend(authors)

# Print the authors
for author in all_authors:
    print(author)


get_all_authors()
print(all_authors[10])
print(len(all_authors))
print(all_authors)

counter=0
for i in range(len(all_authors)):
    if all_authors[i]==None:
      counter=counter+1
print(counter)

#<p class="lx-commentary__meta-reporter gel-long-primer" data-reactid=".18n36quudgy.0.2.0.0.1.0.1.0.$lx-tabs0.0.$lx-commentary.$lx-commentary.0.2.0.0">  Edited by Nadia Ragozhina and Samuel Horti</p>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By Jeremy Bowen</div>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By David Gritten</div>

 Jacqueline Howard
76
[None, ' Lucy Williamson at Al-Shifa hospital, Gaza City', ' David Gritten', ' Anastasia Golubeva and Ido Vock', ' Guy Hedgecoe in Madrid & Paul Kirby in London', ' Yasmin Rufo', ' Max Matza in San Francisco & Gareth Evans in Washington', ' Mike Wendling', ' Ido Vock', None, ' Jacqueline Howard', None, ' Jacqueline Howard', None, ' Michael Race', ' Malu Cursino', ' BBC Verify team', ' Jeremy Bowen', ' Yolande Knell', ' Sam Hancock', ' Anthony Zurcher', ' Jonathan Head', None, None, ' Mike Wendling', ' Vitaly Shevchenko', ' Zoya Mateen', ' Luke Wolstenholme', ' Sahnun Ahmed', None, None, None, None, None, None, ' Riyah Collins', ' Yasmin Rufo', " Gem O'Reilly", None, ' Brandon Drenon', ' Sofia Bettiza', ' Yasmin Rufo', ' Lucy Williamson at Al-Shifa hospital, Gaza City', None, ' Mike Wendling', ' Michael Race', ' Anastasia Golubeva and Ido Vock', ' Guy Hedgecoe in Madrid & Paul Kirby in London', ' Mike Wendling', ' Ido Vock', ' Brandon Drenon', ' Katherine Latham', 

In [11]:
#dictionary with all infos, ordered after attributes
from datetime import datetime
import pytz

germany_timezone = pytz.timezone("Europe/Berlin")


articles_info={}
for i in range(len(news_urls)):
 article_info_i={ 
  "headline": headers[i],
  "url": news_urls[i],
  "date":time[i],
  "authors":[all_authors[i]],
  "ImageURL":imageURL[i],
  "ImageDescription":imageDesc[i],
  "scrapingTimeStamp":(datetime.now(germany_timezone).isoformat()).split(".")[0],
  "contents":text_contents[i]
 }
 articles_info[i]=article_info_i
print(articles_info[20])
#URLId, Headline, Contents, Authors, UploadDate, ReadTime, ImageURL, ImageDescription

{'headline': 'US evangelicals drive Republican support for Israel', 'url': 'https://www.bbc.com/news/world-us-canada-67422238', 'date': '2023-11-15T03:10:43', 'authors': [' Anthony Zurcher'], 'ImageURL': 'https://ichef.bbci.co.uk/news/976/cpsprodpb/1027C/production/_131727166_gettyimages-1782697662-1.jpg', 'ImageDescription': 'People wearing Israeli flags walk towards the Capitol', 'scrapingTimeStamp': '2023-11-16T19:48:08', 'contents': 'US evangelicals drive Republican support for Israel\nPublished\n1 day ago\nShare\nRelated Topics\nIsrael-Gaza war\nIMAGE SOURCE,\nGETTY IMAGES\nBy Anthony Zurcher\nNorth America correspondent\nRepublican support for Israel has been near monolithic since the 7 October attack by Hamas.\nConservatives argue the US is backing a close ally, standing up for the region\'s only democracy and sending a message that terror against civilians will not be tolerated.\nBut there\'s more to it than that.\nEvangelical conservatives are a key part of the Republican part

In [12]:
import psycopg2

In [13]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [14]:
for article in articles_info.values():
    try:
        cursor.execute('''INSERT INTO Articles(URLId, Headline, Content, Authors, UploadTimestamp, ImageURL, ImageDescription,scrapingTimeStamp) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);''', 
                       (article["url"], article["headline"], article["contents"], article["authors"], article["date"], article["ImageURL"], article["ImageDescription"], article["scrapingTimeStamp"])
                       )
    except psycopg2.IntegrityError as e:
        if "duplicate key value violates unique constraint" in str(e):
            print(f"Article with URLId {article['url']} already exists in the database.")




Article with URLId https://www.bbc.co.uk/news/articles/cd1pd66y9k6o already exists in the database.
Article with URLId https://www.bbc.com/news/uk-67436126 already exists in the database.
Article with URLId https://www.bbc.com/news/world_radio_and_tv already exists in the database.
Article with URLId https://www.bbc.com/sounds/play/live:bbc_world_service already exists in the database.
Article with URLId https://www.bbc.com/news/world-65432059 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67434936 already exists in the database.
Article with URLId https://www.bbc.com/news/entertainment-arts-67415437 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67436154 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67437835 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67433963 already exists in the database.
Article wi